<a href="https://colab.research.google.com/github/vutl/Automatic-Speech-Recognition/blob/main/ASR_Project_XLSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
kynthesis_vivos_vietnamese_speech_corpus_for_asr_path = kagglehub.dataset_download('kynthesis/vivos-vietnamese-speech-corpus-for-asr')

print('Data source import complete.')


In [ ]:
!pip install --upgrade transformers datasets jiwer librosa evaluate

In [ ]:
# Import các thư viện
import pandas as pd
import numpy as np
import os
from datasets import load_dataset, Audio
from transformers import (Wav2Vec2ForCTC, Wav2Vec2Processor,
                          Trainer, TrainingArguments, Wav2Vec2CTCTokenizer,
                          Wav2Vec2FeatureExtractor)
import torch
import re
import evaluate  # Thay thế load_metric bằng evaluate

In [ ]:
# Kiểm tra thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Đường dẫn đến dataset VIVOS
train_audio_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/train/waves'
train_prompts_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/train/prompts.txt'

test_audio_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/test/waves'
test_prompts_path = '../input/vivos-vietnamese-speech-corpus-for-asr/vivos/test/prompts.txt'

In [ ]:
# Hàm để đọc file prompts.txt và trả về DataFrame
def load_prompts(prompts_path):
    transcripts = []
    with open(prompts_path, 'r', encoding='utf-8') as f:
        for line in f:
            id, text = line.strip().split(' ', 1)
            transcripts.append({'id': id, 'text': text.lower()})
    return pd.DataFrame(transcripts)

# Tạo DataFrame cho tập train và test
train_transcripts = load_prompts(train_prompts_path)
test_transcripts = load_prompts(test_prompts_path)

In [ ]:
# Thêm đường dẫn âm thanh vào DataFrame
def get_audio_path(audio_base_path, audio_id):
    speaker = audio_id.split('_')[0]
    return os.path.join(audio_base_path, speaker, audio_id + '.wav')

train_transcripts['audio'] = train_transcripts['id'].apply(lambda x: get_audio_path(train_audio_path, x))
test_transcripts['audio'] = test_transcripts['id'].apply(lambda x: get_audio_path(test_audio_path, x))

In [ ]:
# Loại bỏ các ký tự đặc biệt và chuyển văn bản về chữ thường
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

train_transcripts = train_transcripts.apply(remove_special_characters, axis=1)
test_transcripts = test_transcripts.apply(remove_special_characters, axis=1)

In [ ]:
vocab_dict = {"a": 1, "b": 2, "c": 3, "d": 4, "e": 5, "f": 6, "g": 7, "h": 8, "i": 9, "j": 10, "k": 11, "l": 12, "m": 13, "n": 14, "o": 15, "p": 16, "q": 17, "r": 18, "s": 19, "t": 20, "u": 21, "v": 22, "w": 23, "x": 24, "y": 25, "z": 26, "à": 27, "á": 28, "â": 29, "ã": 30, "è": 31, "é": 32, "ê": 33, "ì": 34, "í": 35, "ò": 36, "ó": 37, "ô": 38, "õ": 39, "ù": 40, "ú": 41, "ý": 42, "ă": 43, "đ": 44, "ĩ": 45, "ũ": 46, "ơ": 47, "ư": 48, "ạ": 49, "ả": 50, "ấ": 51, "ầ": 52, "ẩ": 53, "ẫ": 54, "ậ": 55, "ắ": 56, "ằ": 57, "ẳ": 58, "ẵ": 59, "ặ": 60, "ẹ": 61, "ẻ": 62, "ẽ": 63, "ế": 64, "ề": 65, "ể": 66, "ễ": 67, "ệ": 68, "ỉ": 69, "ị": 70, "ọ": 71, "ỏ": 72, "ố": 73, "ồ": 74, "ổ": 75, "ỗ": 76, "ộ": 77, "ớ": 78, "ờ": 79, "ở": 80, "ỡ": 81, "ợ": 82, "ụ": 83, "ủ": 84, "ứ": 85, "ừ": 86, "ử": 87, "ữ": 88, "ự": 89, "ỳ": 90, "ỵ": 91, "ỷ": 92, "ỹ": 93, "|": 0, "<bos>": 94, "<eos>": 95, "<unk>": 96, "<pad>": 97}

In [ ]:
# Lưu vocabulary thành file JSON
import json

with open('vocab.json', 'w', encoding='utf-8') as vocab_file:
    json.dump(vocab_dict, vocab_file, ensure_ascii=False)

# Khởi tạo tokenizer
tokenizer = Wav2Vec2CTCTokenizer("vocab.json",
                                 unk_token="<unk>",
                                 pad_token="<pad>",
                                 word_delimiter_token="|")

In [ ]:
# Khởi tạo feature extractor
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=False
)

# Khởi tạo processor
processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer
)

In [ ]:
# Chuyển đổi DataFrame thành Dataset của Hugging Face
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_transcripts)
test_dataset = Dataset.from_pandas(test_transcripts)

# Chuyển cột 'audio' thành kiểu Audio
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
# Hàm tiền xử lý dữ liệu
def prepare_dataset(batch):
    # Xử lý âm thanh
    audio = batch["audio"]
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    # Xử lý văn bản
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    return batch

# Áp dụng hàm tiền xử lý dữ liệu
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)

In [ ]:
# Khởi tạo Data Collator
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        # Tách inputs và labels
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # Padding inputs
        batch = self.processor.feature_extractor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # Padding labels
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # Thay thế giá trị padding bằng -100
        labels = labels_batch["input_ids"].masked_fill(labels_batch["attention_mask"].ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
# Định nghĩa hàm tính WER
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    # Giải mã dự đoán
    pred_str = processor.batch_decode(pred_ids)

    # Giải mã nhãn
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, group_tokens=False)

    # Tính WER
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [ ]:
# Tải mô hình pre-trained mới (facebook/wav2vec2-large-xlsr-53)
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    activation_dropout=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

# Đóng băng tầng feature extractor
model.freeze_feature_encoder()

# Thiết lập tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./wav2vec2-vivos",
    group_by_length=True,
    per_device_train_batch_size=4,  # Giảm batch size nếu cần
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=10,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
    push_to_hub=False,
    report_to="none",
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [ ]:
# Bắt đầu huấn luyện
trainer.train()

# Lưu mô hình cuối cùng
trainer.save_model("./wav2vec2-vivos")
# Lưu processor
processor.save_pretrained("./wav2vec2-vivos")

In [ ]:
import librosa

# Tải lại processor và mô hình đã huấn luyện
processor = Wav2Vec2Processor.from_pretrained("./wav2vec2-vivos")
model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2-vivos")

# Chuyển mô hình sang chế độ đánh giá
model.eval()

# Chọn một file âm thanh từ tập test
audio_file = test_transcripts['audio'].iloc[0]
print(f"Đang thử nghiệm với file âm thanh: {audio_file}")

# Đọc file âm thanh
audio_input, sampling_rate = librosa.load(audio_file, sr=16000)

# Tiền xử lý âm thanh
input_values = processor(audio_input, sampling_rate=16000, return_tensors="pt").input_values

# Dự đoán
with torch.no_grad():
    logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])

print("Kết quả nhận dạng giọng nói:")
print(transcription)